In [10]:
from lerobot.datasets.lerobot_dataset import LeRobotDataset 
from datasets import load_dataset
from collections import defaultdict
import torch
from huggingface_hub import hf_hub_download
import json
import os



In [18]:
repo_id = "lerobot/svla_so101_pickplace"
OUT_PATH = "svla_so101_actions.pt"

ds = LeRobotDataset(repo_id)
print(ds[0].keys())

dict_keys(['observation.images.up', 'observation.images.side', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'task'])


In [26]:
episode_actions = defaultdict(list)

for i in range(len(ds)):
    sample = ds[i]
    ep = int(sample["episode_index"])
    a  = torch.tensor(sample["action"], dtype=torch.float32)  # (6,)
    episode_actions[ep].append(a)

episodes = []
for ep_id in sorted(episode_actions.keys()):
    ep_tensor = torch.stack(episode_actions[ep_id], dim=0)  # (T_ep, 6)
    episodes.append(ep_tensor)

print(f"{len(episodes)} episodes")
print([ep.shape[0] for ep in episodes])  # lengths


/tmp/ipykernel_47193/2143862697.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a  = torch.tensor(sample["action"], dtype=torch.float32)  # (6,)


50 episodes
[303, 266, 230, 183, 215, 231, 224, 203, 226, 204, 216, 201, 210, 231, 223, 226, 201, 243, 214, 210, 243, 224, 237, 211, 223, 220, 199, 203, 224, 223, 288, 272, 270, 239, 207, 262, 254, 227, 256, 238, 293, 306, 263, 281, 261, 250, 303, 259, 272, 271]


In [27]:
torch.save(episodes, OUT_PATH)